# PRRI: QAnon demographics and attitudes

## Python tools and notebook config

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles
import pytz

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

## Read data

Summary of 2021 findings in four [PRRI surveys](https://www.prri.org/research/the-persistence-of-qanon-in-the-post-trump-era-an-analysis-of-who-believes-the-conspiracies/)

#### Identifying questions

In [5]:
questions_df = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vRErtZoUnAb92p8swbwVnJB0KZD-yw3lWIHWfyBcPffBh2EpXc7vFadF1XYR9_wQuL_0tOn0_AW-v5v/pub?output=xlsx",
    sheet_name="questions",
)

In [6]:
alt.Chart(questions_df).mark_bar().encode(
    x=alt.X("share", stack="normalize", axis=alt.Axis(format="%", tickCount=6)),
    y=alt.Y("question", title=" "),
    color=alt.Color(
        "response",
        title="",
        scale=alt.Scale(
            domain=[
                "Completely agree",
                "Mostly agree",
                "Mostly disagree",
                "Completely disagree",
                "Refused/Don't know",
            ],
            range=["#4b8f84", "#8dc0a1", "#cdc979", "#d29a3a", "#e3e3e3"],
        ),
    ),
    order=alt.Order(
        # Sort the segments of the bars by this field
        "order",
        sort="ascending",
    ),
).properties(width=650, height=100).configure_legend(orient="top")

In [31]:
questions_pivot = questions_df.pivot_table(
    values="share", columns="response", index="question"
).reset_index()

In [33]:
questions_pivot["Completely/mostly agree"] = (
    questions_pivot["Completely agree"] + questions_pivot["Mostly agree"]
)
questions_pivot["Completely/mostly disagree"] = (
    questions_pivot["Completely disagree"] + questions_pivot["Mostly disagree"]
)

In [35]:
questions_pivot.columns

Index(['question', 'Completely agree', 'Completely disagree', 'Mostly agree',
       'Mostly disagree', 'Refused/Don't know', 'Completely/mostly agree',
       'Completely/mostly disagree'],
      dtype='object', name='response')

In [37]:
questions_pivot[
    [
        "question",
        "Refused/Don't know",
        "Completely/mostly agree",
        "Completely/mostly disagree",
    ]
].to_csv("data/processed/qanon_three_questions_stacked_bars.csv", index="False")

---

#### Share of Americans who are QAnon believers, doubters and rejectors

In [7]:
believers_df = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vRErtZoUnAb92p8swbwVnJB0KZD-yw3lWIHWfyBcPffBh2EpXc7vFadF1XYR9_wQuL_0tOn0_AW-v5v/pub?output=xlsx",
    sheet_name="believers",
)

In [8]:
believers_df

,Date,QAnon believers,doubters,rejectors
0,2021-03-01,0.14,0.46,0.40
1,2021-07-01,0.16,0.49,0.35
2,2021-09-01,0.17,0.48,0.35
3,2021-10-01,0.17,0.49,0.34


In [9]:
believers_df_melted = believers_df.melt(
    id_vars="Date", value_vars=["QAnon believers", "doubters", "rejectors"]
)

In [10]:
believers_df_melted["date"] = believers_df_melted.Date.astype(str)

In [11]:
alt.Chart(believers_df_melted).mark_line().encode(
    x=alt.X("date:O", axis=alt.Axis(format="", tickCount=5), title=""),
    y=alt.Y("value", title=" ", axis=alt.Axis(tickCount=4, format="%")),
    color=alt.Color(
        "variable",
        title=" ",
        scale=alt.Scale(
            domain=[
                "QAnon believers",
                "doubters",
                "rejectors",
            ],
            range=["#00d4d8", "#cdc979", "#d95f1a"],
        ),
    ),
).properties(
    width=650, height=300, title="Attitudes about QAnon in 2021"
).configure_legend(
    orient="top", symbolType="stroke"
)

alt.Chart(...)

---

#### Share of QAnon believers by political party

In [12]:
party_df = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vRErtZoUnAb92p8swbwVnJB0KZD-yw3lWIHWfyBcPffBh2EpXc7vFadF1XYR9_wQuL_0tOn0_AW-v5v/pub?output=xlsx",
    sheet_name="parties",
)

In [13]:
party_df

,Party,Percent of QAnon believers who are
0,Republican,43.0
1,Independent,27.0
2,Democrat,19.0
3,Other,12.0


In [14]:
alt.Chart(party_df).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="Percent of QAnon believers who are", type="quantitative"),
    color=alt.Color(
        field="Party",
        type="nominal",
        scale=alt.Scale(
            domain=[
                "Republican",
                "Democrat",
                "Independent",
                "Other",
            ],
            range=["#c62222", "#1851ac", "#9b69c7", "#e3e3e3"],
        ),
    ),
).properties(width=200, height=200)

In [27]:
party_df.T

,0,1,2,3
Party,Republican,Independent,Democrat,Other
Percent of QAnon believers who are,43.0,27.0,19.0,12.0


#### Share of QAnon believers, by education level

In [15]:
education_df = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vRErtZoUnAb92p8swbwVnJB0KZD-yw3lWIHWfyBcPffBh2EpXc7vFadF1XYR9_wQuL_0tOn0_AW-v5v/pub?output=xlsx",
    sheet_name="education",
)

In [16]:
education_df

,Level,Percent of QAnon believers who have a
0,High school degree or less,57.0
1,Some college but no degree,29.0
2,Four-year degree,9.0
3,Post-graduate degree,5.0


In [17]:
alt.Chart(education_df).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="Percent of QAnon believers who have a", type="quantitative"),
    color=alt.Color(
        field="Level",
        type="nominal",
    ),
).properties(width=200, height=200)

alt.Chart(...)